## Analyse bornes de recharges publiques pour voitures électriques
source:
https://www.data.gouv.fr/fr/datasets/bornes-de-chargement-publiques-pour-voitures-electriques-du-plusieurs-operateurs/

In [1]:
import requests
import pandas as pd
import numpy as np
import leafmap
import re
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [2]:
#https://www.data.gouv.fr/fr/datasets/bornes-de-chargement-publiques-pour-voitures-electriques-du-plusieurs-operateurs/
url = "https://www.data.gouv.fr/fr/datasets/r/b11113db-875d-41c7-8673-0cf8ad43e917"
nom_fichier = "Src_bornes_gouv.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [3]:

    
url2 = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
nom_fichier = "Src_depart_gouv.geojson"

response = requests.get(url2)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [4]:
data_dpt=gpd.read_file('Src_depart_gouv.geojson')

In [5]:
df = pd.read_csv('Src_bornes_gouv.csv')

df.sample(10)

,contact_amenageur,contact_operateur,nom_amenageur,nom_operateur,telephone_operateur,siren_amenageur,raccordement,nom_enseigne,id_station_itinerance,id_station_local,nom_station,implantation_station,adresse_station,coordonneesXY,nbre_pdc,id_pdc_itinerance,id_pdc_local,puissance_nominale,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,gratuit,tarification,paiement_autre,paiement_acte,paiement_cb,condition_acces,reservation,horaires,accessibilite_pmr,restriction_gabarit,station_deux_roues,date_mise_en_service,observations,date_maj,cable_t2_attache
3963,kontakt@lidl.de,kontakt@lidl.de,NaN,NaN,+(33)-(800)-900343,NaN,NaN,Lidl GUICHEN Joliot Curie,FRLDLE0000196,FRLDLE0000196,Lidl GUICHEN Joliot Curie,Voirie,"Rue Joliot Curie 2, 35580 Guichen","[47.97307600,-1.81350600]",3,FRLDLE00001969,FRLDLE00001969,22,False,True,False,False,False,False,NaN,NaN,True,False,Accès libre,True,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",Accessibilité inconnue,inconnu,False,2023-02-14,NaN,2023-10-02,False
5043,klantenservice@newmotion.com,klantenservice@newmotion.com,NaN,NaN,+(33)-(9)-69366018,NaN,NaN,09910787,FRTNME09910787,FRTNME09910787,09910787,Voirie,"36 Rue des Bellangeres, 28630 Le Coudray","[48.41701300,1.49050300]",2,FRTNME099107871,FRTNME099107871,2,True,False,False,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2020-10-31,NaN,2023-10-03,False
7187,info@chargeguru.com,info@chargeguru.com,NaN,NaN,+33176341178,NaN,NaN,Volkswagen ChateauBernard - 16100 - 1,FRCG0E00088,FRCG0E00088,Volkswagen ChateauBernard - 16100 - 1,Station dédiée à la recharge rapide,"13 Rue Jean-Loup Chrétien, 16100 Châteaubernard","[45.67972400,-0.29453100]",2,FRCG0E000889,FRCG0E000889,150,False,False,True,False,False,False,NaN,NaN,True,False,Accès libre,True,24/7,Accessibilité inconnue,inconnu,False,2023-08-26,NaN,2023-09-27,False
6246,kontakt@lidl.de,kontakt@lidl.de,NaN,NaN,+(33)-(800)-900343,NaN,NaN,Lidl KINGERSHEIM Mulhouse,FRLDLE0000027,FRLDLE0000027,Lidl KINGERSHEIM Mulhouse,Voirie,"Faubourg de Mulhouse, 68260 Kingersheim","[47.78942600,7.33576500]",2,FRLDLE00000276,FRLDLE00000276,22,False,True,False,False,False,False,NaN,NaN,True,False,Accès libre,True,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",Accessibilité inconnue,inconnu,False,2022-06-24,NaN,2023-10-03,False
9679,support@modulo-energies.fr,support@modulo-energies.fr,Modulo Energies,Modulo,0800 37 37 40,841376734.0,NaN,MONTOIRE SUR LE LOIR 4931 - Rue Marecot,FRS41E493,FRS41E493,MONTOIRE SUR LE LOIR 4931 - Rue Marecot,Voirie,"Rue Marecot, 41800 MONTOIRE SUR LE LOIR","[47.75157900,0.86145800]",2,FRS41E4932,FRS41E4932,18,False,True,False,False,False,False,NaN,True,True,False,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2019-03-18,NaN,2023-10-02,False
10,support@modulo-energies.fr,support@modulo-energies.fr,Modulo Energies,Modulo,0800 37 37 40,841376734.0,NaN,COURS-LES*BARRES - Grande Rue - 181472,FRS18E18147,FRS18E18147,COURS-LES*BARRES - Grande Rue - 181472,Voirie,"Grande Rue, 18320 Cours-les-Barres","[47.02565600,3.03166000]",2,FRS18E181472,FRS18E181472,18,True,True,False,False,False,False,NaN,True,True,False,Accès réservé,False,24/7,Accessibilité inconnue,inconnu,False,2019-11-08,NaN,2023-10-02,False
4115,info@allego.eu,info@allego.eu,NaN,NaN,+(33)-(800)-78192,NaN,NaN,Allego Carrefour Romorantin,FRALLEGO900556,FRALLEGO900556,Allego Carrefour Romorantin,Voirie,"Avenue de Villefranche_ZAC de Plaisance, 41200...","[47.33774500,1.74848800]",11,FRALLEGO9005562,FRALLEGO9005562,22,False,True,False,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2022-09-06,NaN,2023-10-02,False
8265,support@modulo-energies.fr,support@modulo-energies.fr,Modulo Energies,Modulo,0800 37 37 40,841376734.0,NaN,LAMBLORE - Le Plessis -176251,FRS28E17625,FRS28E17625,LAMBLORE - Le Plessis -176251,Voirie,"Le Plessis, 28340 Lamblore","[48.62729100,0.91635800]",2,FRS28E176251,FRS28E176251,18,

In [6]:
df[['x', 'y']] = df['coordonneesXY'].str.strip('[]').str.split(',', expand=True).astype(float)

def extraire_cp(texte):
    match = re.search(r'\d{5}', texte)
    if match:
        chiffres = match.group()
        return chiffres[:2]  # Prendre les deux premiers chiffres
    else:
        return None

# Appliquez la fonction à la colonne 'Adresse' du DataFrame
df['Departement'] = df['adresse_station'].apply(extraire_cp)



In [7]:
df_region = pd.read_csv('POP_donnees_communes.csv',sep=";")
df_region=df_region[['REG', 'CODDEP',  'COM', 'PTOT']]
df_region = df_region.groupby('CODDEP').sum().reset_index()
df_region

,CODDEP,PTOT
0,01,668565
1,02,543368
2,03,344922
3,04,168867
4,05,145791
...,...,...
95,95,1259985
96,971,389995
97,972,369406
98,973,284085


In [8]:
data_dpt

,code,nom,geometry
0,02,Aisne,"POLYGON ((3.17270 50.01200, 3.18220 50.01234, ..."
1,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ..."
2,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240..."
3,15,Cantal,"POLYGON ((2.50841 45.47850, 2.52444 45.48070, ..."
4,28,Eure-et-Loir,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ..."
...,...,...,...
91,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87588 48.10943, ..."
92,53,Mayenne,"POLYGON ((-0.86036 48.50146, -0.84611 48.49828..."
93,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.45556 49.48093, 5.44953 49.4..."
94,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.87973 46.97580..."


In [9]:
df_count_stat = df.groupby('Departement')['adresse_station'].count()
df_region = pd.merge(df_region,df_count_stat,left_on='CODDEP',right_on='Departement')
df_region['Nb stat pour 100khab'] = round((df_region['adresse_station']*100000)/df_region['PTOT'])
df_region

,CODDEP,PTOT,adresse_station,Nb stat pour 100khab
0,01,668565,102,15.0
1,02,543368,70,13.0
2,03,344922,55,16.0
3,04,168867,28,17.0
4,05,145791,15,10.0
...,...,...,...,...
89,91,1316011,145,11.0
90,92,1640721,41,2.0
91,93,1651739,48,3.0
92,94,1415845,70,5.0


In [10]:
data_dpt=pd.merge(data_dpt,df_region,left_on='code',right_on='CODDEP',suffixes=False)
data_dpt


,code,nom,geometry,CODDEP,PTOT,adresse_station,Nb stat pour 100khab
0,02,Aisne,"POLYGON ((3.17270 50.01200, 3.18220 50.01234, ...",02,543368,70,13.0
1,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...",10,317366,58,18.0
2,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240...",14,709064,140,20.0
3,15,Cantal,"POLYGON ((2.50841 45.47850, 2.52444 45.48070, ...",15,149664,13,9.0
4,28,Eure-et-Loir,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...",28,441931,233,53.0
...,...,...,...,...,...,...,...
89,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87588 48.10943, ...",41,337975,287,85.0
90,53,Mayenne,"POLYGON ((-0.86036 48.50146, -0.84611 48.49828...",53,316615,49,15.0
91,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.45556 49.48093, 5.44953 49.4...",54,745453,126,17.0
92,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.87973 46.97580...",79,385150,90,23.0


In [11]:
data_dpt.to_file("Src_dpt_final.geojson", driver='GeoJSON')

In [12]:
df_map = df[['nom_operateur','raccordement', 'nom_enseigne', 'id_station_itinerance',
       'nom_station', 'implantation_station',
       'adresse_station','nbre_pdc', 'puissance_nominale','gratuit', 'tarification', 'paiement_autre', 'paiement_acte',
       'paiement_cb','restriction_gabarit', 'station_deux_roues',
       'date_mise_en_service', 'observations', 'date_maj',
       'x', 'y', 'Departement']]

In [13]:


ma_carte = leafmap.Map(center=[45.76077534099906, 4.833680068867414],zoom=12)

ma_carte.add_data('Src_dpt_final.geojson', column='Nb stat pour 100khab' ,scheme='Quantiles',legend_title="Bornes pour 100khab", cmap='Blues',layer_name='Departements')
        
ma_carte.add_points_from_xy(data=df_map, x='y', y='x', radius=10)
#ma_carte.add_data(data=df_region, column='Nb stat pour 100khab', scheme='Quantiles', cmap='Blues', legend_title='Population'
#) 


In [14]:
ma_carte

Map(center=[45.76077534099906, 4.833680068867414], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [15]:
ma_carte.to_html("output.html")

In [16]:
df_region.sort_values(by='Nb stat pour 100khab')

,CODDEP,PTOT,adresse_station,Nb stat pour 100khab
90,92,1640721,41,2.0
91,93,1651739,48,3.0
73,75,2182174,59,3.0
63,65,235709,9,4.0
92,94,1415845,70,5.0
...,...,...,...,...
26,28,441931,233,53.0
53,55,189535,143,75.0
17,18,309523,250,81.0
39,41,337975,287,85.0
